In [ ]:
import calendar
import random

ausgaben = []

for year in range(1872, 1930 + 1):
    for month in range(1, 12 + 1):
        _, n_days = calendar.monthrange(year, month)
        for day in range(1, n_days + 1):
            for ausgabe in ('M', 'A'):
                ausgaben.append((year, month, day, ausgabe))
            
random.shuffle(ausgaben)

In [ ]:
len(ausgaben)

In [ ]:
# speicherbedarf in mb
(len(ausgaben) * 20 * 2) / (1024 * 1024)

In [ ]:
ausgaben[:10]

In [ ]:
from pathlib import Path
import requests
from tqdm import tqdm
import random
import xml.etree.ElementTree as ET

cache_path = Path('/Users/offline/Dropbox/Bernhards Bereiche/BBZ/cache')

def download(url, path):
    r = requests.get(url, stream=True)

    total_size = int(r.headers.get('content-length', 0))
    block_size = 1024

    with tqdm(total=total_size, unit='iB', unit_scale=True, desc=url) as t:
        with open(path, 'wb') as f:
            for data in r.iter_content(block_size):
                t.update(len(data))
                f.write(data)

    if total_size != 0 and t.n != total_size:
        raise RuntimeError("%d != %d" % (t.n, total_size))

def zefys_id(ausgabe, seite=0):
    jahr, monat, tag, typ = ausgabe
    return 'SNP2436020X-%04d%02d%02d-%d-%d-0-0' % (jahr, monat, tag, 0 if typ == 'M' else 1, seite)

def download_page(ausgabe, seite=1):
    p = cache_path / 'jpg' / ('%s.jpg' % zefys_id(ausgabe, seite))
    if not p.is_file():
        urls = bbz_page_urls(ausgabe)
        download(urls[seite - 1], p)
    return p

def download_random_page(ausgabe):
    urls = bbz_page_urls(ausgabe)
    return download_page(ausgabe, random.randint(1, len(urls)))

def bbz_xml(ausgabe):
    p = cache_path / 'xml' / ('%s.xml' % zefys_id(ausgabe))
    if not p.is_file():
        download('http://content.staatsbibliothek-berlin.de/zefys/%s.xml' % zefys_id(ausgabe), p)
    return ET.parse(p)

def bbz_page_urls(ausgabe):
    tree = bbz_xml(ausgabe)
    root = tree.getroot()
    loc = root.findall('.//{http://www.loc.gov/METS/}fileGrp[@USE="MAX"]/{http://www.loc.gov/METS/}file/{http://www.loc.gov/METS/}FLocat')
    return [l.attrib['{http://www.w3.org/1999/xlink}href'] for l in loc]

def download_stichprobe(ausgaben):
    for a in ausgaben:
        try:
            download_random_page(a)
        except:
            print("failed to fetch ausgabe", a)

